### Deploy Web App on Azure Container Services (AKS)
In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored.
    
The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create resource group and create AKS](#section3)
* [Connect to AKS](#section4)
* [Deploy our app](#section5)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

In [1]:
import os
import json
from testing_utilities import write_json_to_file
%load_ext dotenv

<a id='section1'></a>
## Setup
Below are the various name definitions for the resources needed to setup AKS.

In [6]:
%%writefile --append .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use 
selected_subscription = "Team Danielle Internal"

# Resource group, name and location for AKS cluster.
resource_group = "mabouaks" 
aks_name = "mabouaks"
location = "eastus"

Appending to .env


In [10]:
%dotenv -o
image_name = os.getenv('docker_login') + os.getenv('image_repo')

<a id='section2'></a>
## Azure account login
If you are not already logged in to an Azure account, the command below will initiate a login. It will pop up a browser where you can select an Azure account.

In [11]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

Name                                            CloudName    SubscriptionId                        State    IsDefault
----------------------------------------------  -----------  ------------------------------------  -------  -----------
Boston DS Dev                                   AzureCloud   0ca618d2-22a8-413a-96d0-0f1b531129c3  Enabled  False
Solution Template Testing                       AzureCloud   3bcfa59c-82a0-44f9-ac08-b3479370bace  Enabled  False
DEMO - how RepDemo are you                      AzureCloud   fe4d94f0-dc5b-4c09-9b85-863413b0192b  Enabled  False
Microsoft Azure Internal - Demos                AzureCloud   b3a823e7-7472-4111-becc-9a211a558a05  Enabled  False
Azure Stack Diagnostics CI and Production VaaS  AzureCloud   a8183b2d-7a4c-45e9-8736-dac11b84ff14  Enabled  False
Azure Cat E2E                                   AzureCloud   fc4ea3c9-1d30-4f18-b33b-7404e7da0123  Enabled  False
Core-ES-BLD                                     AzureCloud   54e18c35-3863-4a1

In [12]:
!az account set --subscription "$selected_subscription"

In [13]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "mabou@microsoft.com",
    "type": "user"
  }
}


In [14]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [15]:
!az provider show -n Microsoft.ContainerService

{
  "authorization": {
    "applicationId": "7319c514-987d-4e9b-ac3d-d38c4f427f4c",
    "managedByRoleDefinitionId": "8e3af657-a8ff-443c-a75c-2fe8c4bcb635",
    "roleDefinitionId": "1b4a0c7f-2217-416f-acfa-cf73452fdc1c"
  },
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/providers/Microsoft.ContainerService",
  "namespace": "Microsoft.ContainerService",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "aliases": null,
      "apiVersions": [
        "2017-07-01",
        "2017-01-31",
        "2016-09-30",
        "2016-03-30"
      ],
      "capabilities": "None",
      "locations": [
        "Japan East",
        "Central US",
        "East US 2",
        "Japan West",
        "East Asia",
        "South Central US",
        "Australia East",
        "Australia Southeast",
        "Brazil South",
        "Southeast Asia",
        "West US",
        "North Central US",
        "West Europe",
        "North Europe",
        "East US",
        "UK West"

<a id='section3'></a>
## Create resource group and create AKS

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [16]:
 !az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/mabouaks",
  "location": "eastus",
  "managedBy": null,
  "name": "mabouaks",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier. This can take up to 15 minutes.

In [ ]:
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

SSH key files '/home/mabou/.ssh/id_rsa' and '/home/mabou/.ssh/id_rsa.pub' have been generated under ~/.ssh to allow SSH access to the VM. If using machines without permanent storage like Azure Cloud Shell without an attached file share, back up your keys to a safe location


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [ ]:
!sudo az aks install-cli

<a id='section4'></a>
## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [ ]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Let's verify connection by listing the nodes.

In [ ]:
!kubectl get nodes

Let's check the pods on our cluster.

In [ ]:
!kubectl get pods --all-namespaces

<a id='section5'></a>
## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.


In [ ]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                            "mountPath": "/usr/local/nvidia",
                            "name": "nvidia"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [ ]:
write_json_to_file(app_template, 'az-dl.json') # We write the service template to the json file

In [ ]:
write_json_to_file(service_temp, 'az-dl.json', mode='a') # We add the loadbelanacer template to the json file

Let's check the manifest created.

In [ ]:
!cat az-dl.json

Next, we will use kubectl create command to deploy our application.

In [ ]:
!kubectl create -f az-dl.json

Let's check if the pod is deployed.

In [ ]:
!kubectl get pods --all-namespaces

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [ ]:
!kubectl get events

In [ ]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

It can take a few minutes for the service to populate the EXTERNAL-IP field. This will be the IP you use to call the service. You can also specify an IP to use please see the AKS documentation for further details.

In [ ]:
!kubectl get service azure-dl

Next, we will [test our web application](05_TestWebApp.ipynb) deployed on AKS. 